In [86]:
import pandas as pd
import pymorphy2
from nltk.stem import WordNetLemmatizer

In [87]:
train = pd.read_csv("C:\\Users\In9.var\Desktop\Projects\data1\\train_clever.csv", sep=",")
test = pd.read_csv("C:\\Users\In9.var\Desktop\Projects\data1\\test_clever.csv", sep=",")
train=train.drop(["ID"], axis=1)
test=test.drop(["ID"], axis=1)
train

,Answer,Question
0,0,Крупнейшим производителем кукурузы является:
1,0,Кто первый в РОССИЮ завёз картошку ?
2,0,Кто из мореплавателей сделал первое кругосветн...
3,0,Как зовут героя серии романов Бариса Акунина?
4,0,Какая сборная выйграла чемпионат мира по футбо...
...,...,...
19995,0,В каком году изобрели компьютерную мышь?
19996,0,Каким прозвищем удостоен известный американски...
19997,0,Дата начала Второй Великой Отечественной войны
19998,0,В какой Республике находиться город Стерлитамак


In [88]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Answer    20000 non-null  int64 
 1   Question  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


In [89]:
s = sum(train["Answer"].values) # обидно
s

2115

In [90]:
train.shape

(20000, 2)

In [91]:
morph = pymorphy2.MorphAnalyzer()

In [92]:
train["Question"] = train["Question"].str.lower()
test["Question"] = test["Question"].str.lower()

In [93]:
arr = []
for i in range(train.shape[0]):
    if train["Answer"][i] == 0:
        if s != 0:
            s-=1
        else:
            arr.append(i)
train = train.drop(arr)
train

,Answer,Question
0,0,крупнейшим производителем кукурузы является:
1,0,кто первый в россию завёз картошку ?
2,0,кто из мореплавателей сделал первое кругосветн...
3,0,как зовут героя серии романов бариса акунина?
4,0,какая сборная выйграла чемпионат мира по футбо...
...,...,...
19962,1,кто сыграл энди дюфрейна в фильме «побег из шо...
19970,1,кого свергла екатерина ii?
19980,1,сколько игроков в футбольной команде?
19984,1,"как называется бурятское блюдо, похожее на манты?"


In [94]:
punctuation_signs = list("?:!.,;")
for punct_sign in punctuation_signs:
    train['Question'] = train['Question'].str.replace(punct_sign, '')
    test['Question'] = test['Question'].str.replace(punct_sign, '')

In [95]:
def func(df, col):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_text_list = []
    for row in range(len(df)):
        lemmatized_list = []
        text = df.iloc[row][col]
        text_words = text.split(" ")
        for word in text_words:
            word_norm = morph.parse(word)[0][2]
            lemmatized_list.append(word_norm)
        lemmatized_text = " ".join(lemmatized_list)
        lemmatized_text_list.append(lemmatized_text)
    return lemmatized_text_list


train["Question"] = func(train, "Question")
train
test["Question"] = func(test, "Question")

In [96]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['Question'], 
                                                    train['Answer'], 
                                                    test_size=0.2, 
                                                    random_state=8)
print(X_train, X_test, y_train, y_test, sep="\n\n\n")

1997                    какой из этот язык не существовать
16981                 как называться водный оболочка земля
15934                       кто из указанный актёр старший
1462         в чем состоять основный заслуга чарльз дарвин
17736    как звать единственный в история футбол вратар...
                               ...                        
2184     сколько раз старик из сказка а с пушкин вызыва...
2181       как по-другому называться соус на основа молоко
2637                 в какой год быть основать город тверь
2033                          самый близкий к земля звезда
1364                   сколько планета в солнечный система
Name: Question, Length: 3384, dtype: object


3440              как называться бросок сверху в баскетбол
16492    как называться способность поглощение влажност...
1898             в какой месяц справлять новый год в китай
13003    сборная какой страна выиграть первый официальн...
671      новый сезон какой сериал не выходить в сентябр...
          

In [140]:
from sklearn.feature_extraction.text import TfidfVectorizer

ngram_range = (1, 2)

min_df = 10
max_df = 1.
max_features = 300

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)
ans_features = tfidf.fit_transform(test["Question"]).toarray()
features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)
feature_names = tfidf.get_feature_names()
train_ready = pd.DataFrame(features_train, columns=feature_names)
train_ready

(3384, 300)
(846, 300)


,2018,автомобиль,автор,актёр,александр,английский,бог,больший,большой,брат,...,что,что из,что такой,чтобы,элемент,это,этот,являться,являться самый,язык
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.29014,0.0,0.0,0.0,0.0,0.430684
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.708379,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3379,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
3380,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
3381,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
3382,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000


In [141]:
from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier(max_depth=10,random_state=42)
model.fit(features_train, labels_train)
model.score(features_test, labels_test)

0.5543735224586288

In [144]:
y_predicted_test = model.predict(ans_features)
y_test_pred = pd.DataFrame(y_predicted_test, columns=['Answer'])
y_test_pred.reset_index(inplace=True)
y_test_pred.to_csv('C:\\Users\In9.var\Desktop\Projects\data1\submition_test_Igor.csv', index=False)
